### Finetuning YOLOv11 with Buffalo and Camel labels.
For installation and setup, view the [README](./README.md) before running this.

As mentioned in the README, we will be using Roboflow for labelling our dataset.